In [3]:
# !pip install -q -U keras-tuner
import tensorflow as tf
from tensorflow import keras

import IPython
import kerastuner as kt

C:\Users\user\AppData\Local\Temp\ipykernel_11956\2498043268.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


## Download Dataset

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

In [6]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

    return model

## Hypertuning

In [7]:
tuner = kt.Hyperband(model_builder,
                        objective = 'val_accuracy', 
                        max_epochs = 10,
                        factor = 3,
                        directory = 'my_dir',
                        project_name = 'intro_to_kt')

In [8]:
# 하이퍼파라미터 검색을 실행하기 전에 훈련 단계가 끝날 때마다 훈련 결과를 지우도록 콜백을 정의
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [9]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 37s]
val_accuracy: 0.8575999736785889

Best val_accuracy So Far: 0.8910999894142151
Total elapsed time: 00h 10m 33s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer
is 0.001.



In [10]:
# Build the model with the optimal hyperparameters and train it on the data
# retrain
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4734 - accuracy: 0.8307 - val_loss: 0.4290 - val_accuracy: 0.8447
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3597 - accuracy: 0.8681 - val_loss: 0.3966 - val_accuracy: 0.8516
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3230 - accuracy: 0.8820 - val_loss: 0.3572 - val_accuracy: 0.8724
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3008 - accuracy: 0.8895 - val_loss: 0.3462 - val_accuracy: 0.8741
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2840 - accuracy: 0.8947 - val_loss: 0.3523 - val_accuracy: 0.8737
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2683 - accuracy: 0.9003 - val_loss: 0.3426 - val_accuracy: 0.8734
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2542 - accuracy: 0.9048 - val_loss: 0.3372 - val_accuracy: